<a href="https://colab.research.google.com/github/LevWilliams/PoliticalClassificationPOSTBERT/blob/master/PNClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modules and Libraries

In [12]:
## importing packages
import gc
import os
import random
!pip install transformers
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K

from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

TensorFlow version: 2.3.0
Transformers version: 3.0.2


Uploading Data

In [13]:
PolySocMed = pd.read_csv("political_social_media.csv", encoding = 'iso-8859-1')

In [14]:
PolySocMed.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,message:confidence,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,1.0,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,1.0,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,1.0,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,1.0,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,1.0,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


In [15]:
TrainAndValDat = PolySocMed[['text','bias']]
TrainAndValDat.head()

,text,bias
0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,partisan
1,VIDEO - #Obamacare: Full of Higher Costs and ...,partisan
2,Please join me today in remembering our fallen...,neutral
3,RT @SenatorLeahy: 1st step toward Senate debat...,neutral
4,.@amazon delivery #drones show need to update ...,partisan


In [17]:
TrainAndValDat['bias']

0       partisan
1       partisan
2        neutral
3        neutral
4       partisan
          ...   
4995    partisan
4996    partisan
4997     neutral
4998     neutral
4999     neutral
Name: bias, Length: 5000, dtype: object

Tokenizing the data

In [16]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("albert-xxlarge-v2")
			
model = AutoModelWithLMHead.from_pretrained("albert-xxlarge-v2")
			

In [18]:
#returns tokenized data
def tokenize_sentences(sentences, tokenizer, max_seq_len = 100):
    tokenized_sentences = []
    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens = True, max_length = max_seq_len)
        tokenized_sentences.append(tokenized_sentence)
    return tokenized_sentences

def preprocessPoliticalData():
    print("Loading data...")
    x_text = TrainAndValDat['text']
    y = TrainAndValDat['bias']
    tokenizer =  AutoTokenizer.from_pretrained("albert-xxlarge-v2", do_lowercase=True)

    x_train = tokenize_sentences(x_text, tokenizer)
    x = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(x_train, 100, padding='post', truncating='post')))

    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_train = x[shuffle_indices]
    y_train = y[shuffle_indices]

    del x, y
    return x_train, y_train

In [ ]:
x_train, y_train =